# Building a Collocation Dictionary

English being my second language, I regularly encounter situations in which I am unable to decide whether I am using an appropriate adjective or adverb, pre- or posposition. One of my favourite places to turn to is http://www.ozdic.com/, an online GUI for the Oxford Collocations Dictionary (OCD). Collocations are sequences of word that co-occur regularly. Learning collocations helps language students sound more authentic. The OCD has been compiled by hand since the late 1950s by language experts and have been updated regularly to reflect changes in linguistic trends.

Working on my original project idea of buliding an n-gram based 'intrinsic plagiarism detector' (which I deemed too difficult a task to do, but will attempt during my summer break), I came upon the idea that n-grams would be ideal to build my own collocation dictionary. The idea is very simple:

1. Given a corpus of text, construct frequency tables for a a modified verison of n-grams, called skip-grams (with a linguistically justified max-skip parameter).
2. Use these frequency tables to create Markov-probability matrices.
3. Given an input word, use Markov matrices to retrieve most likely pre- and post postions of a word.

# Skip Grams

The first step is the construction of skip-grams. Skip Grams are just a slight modification of n-grams. Unlike n-grams, where we want to find $n$ contiguous words from a sequence of words, in skip-grams words do not need to be contiguous. For example the tri-grams for the text "English is my second language" tri-grams would be:

1. "English is my"
2. "is my second"
3. "my second language"

Where the skip-grams of length 3 are much more numerous:

1. "English is my"
2. "English is second"
3. "English is language"
4. "is my second"
5. "is my language"
6. Etc., there exists altogether $k \choose n$ skip-grams, where $k$ is the length of the total sequence and $n$ the length of the skip-gram. In this case ${5 \choose 3}=10$ skip-grams.

Skip-grams are useful as they allow us to take a larger sample of the corpus that can ignore some of the "useless" words such as connectives and common pre or post positions (e.g. the, an, and, or, etc.).

## A dynamic solution

While the runtime complexity of finding all n-grams in a sequence of $N$ words is $O(N)$, as illustrated above, finding skip-grams will optimally take the runtime of the number of $n$-length skip-grams for a sequence of $k$ words, thus $O({k \choose n})$. In order to achieve this complexity however, as opposed to the suboptimal brute force solution of $O(k^n)$ we must devise a dynamic solution.

Also for practical purposes, we will want to limit the number of skips when constructing our skip-grams. Then the optimal substructure for skip-grams of length $n$ and maximum skip of $m$ can be defined as follows. Let $S[p][q]$ be a set of skip-grams of length $q$ (with max skip of $m$) starting at the position $p$ in the input sequence. In order to generate all skip grams of length $q+1$, ending at position k, we can define :

$S[p][q+1] = S[p] + S[p+1][q] + S[p+2][q] + ... S[p+m+1][q]$.

Which means that we first need to take the word at the $p^{th}$ position, and then add all the skip grams of length $q$ startting from $p+1, p+2 ...,p+m+1$. This allows us to define a recursive relation for the subproblem (see below) and memoize our intermediate solutions to reduce the algorithm's runtime complexity.

# Implementation

In [521]:
'''Load corpus.'''
import collections
import re

#read in file
f=open('ulysses.txt',encoding="utf8") # i used james joyce's ulysses
txt=f.read()
f.close()
print ('This book is', txt[:50] , "it is",len(txt), "words long.")

#split by words
words = re.split('[^A-Za-z]+', txt.lower())
words = list(filter(None, words)) # Remove empty strings

This book is ﻿
The Project Gutenberg EBook of Ulysses, by James it is 1539145 words long.


In [514]:
'''
Dynamic Construction of skip-grams. 

'''
def skip_grams_recursion(words, p, n, m, memoi):
    if n == 1:
        #base case, if skip-gram only len 1
        memoi[(p, n)] = [words[p]]
    else:
        # subproblem S[p][q+1]=S[p] + S[p+1][q] + S[p+2][q] + ... S[p+m+1][q]
        current_skipgram = []
        for idx in range(1, m + 2):
            if p + idx < len(words) and n > 1:
                if (p + idx, n - 1) in memoi:
                    current_skipgram += memoi[(p + idx, n - 1)]
                else:
                    #recursive call for next skip-gram segment
                    current_skipgram += skip_grams_recursion(words, p + idx, n - 1, m, memoi)
                    
        new_skipgram = [words[p] + " " + x for x in current_skipgram]
        memoi[(p, n)] = new_skipgram
    return memoi[(p, n)]

def skip_grams(words, n, m):
    memoi = collections.defaultdict(list) #use defaultdict to prevent key-value errors
    skiplist = []
    for idx in range(len(words) - n + 1):
        skiplist += skip_grams_recursion(words, idx, n, m, memoi)
    return skiplist

'''
Construction of dictionary of skip-gram tuples and their frequency.
'''
def skip_to_freq(words,n,m):
    #create a dictionary of frequencies
    out=dict()
    skips=generate_ngram(words,2)
    
    # Populate skip-gram frequency dictionary
    for i in range(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if key in out:
            out[key] += 1
        else:
            out[key] = 1
    return out
'''
Construct a Markov transition matrix from frequencies.
'''

def freq_to_markov(words,n=2,m=0):
    #dict of all words transformed to a list
    sdict=skip_to_freq(words,1,0)
    sdict=list(sdict.keys())
    
    #dict of bigram frequencies
    bfreq=skip_to_freq(words,2,1)
    
    #dimensions of markov matrix
    dim=len(sdict)
    
    #initialise markov matrix
    M = [[0 for x in range(dim)] for y in range(dim)]
    
    #fill out matrix
    for i in range(dim):
        for j in range(dim):
            M[i][j]=bfreq.get(sdict[i]+sdict[j])
    
    return M

M=freq_to_markov(corpus) #construct Markov matrix

In [578]:
'''
Find the collocation of a given string using the skip-gram Markov matrices.

'''

def collocations(M,string,corpus,n=2,m=2):    
    #dict of corpus words
    sdict=skip_to_freq(words,1,0)
    sdict=list(sdict.keys())

    #check if word in corpus
    if (string,) not in sdict:
        raise ValueError('Word is not in corpus. Try another one!')
    
    #find index of element
    idx=sdict.index((string,))
    
    #find five most likely post-postitions to word:
    #sort idx^th column
    column=[i[idx] for i in M]
    for i in range(len(column)):
        if column[i] is None:
            column[i]=0

    top_post=[]
        
    for i in range(5):
        top=column.index(max(column))
        top_post.append(sdict[top])
        column[top]=0
    
    #find five most likely pre-postitions to word:
    #sort idx^th row
    row=M[idx]
    for i in range(len(row)):
        if row[i] is None:
            row[i]=0

    top_pre=[]
        
    for i in range(5):
        top=row.index(max(row))
        top_pre.append(sdict[top])
        row[top]=0
        
    #define "useless" collocations
    useless=['the','and','an','a','but','or', 'a ', 'of ']
    
    #remove "useless" words from top collocations
    for word in useless:
        while (word,) in top_post:
            top_post.remove((word,))
        while (word,) in top_pre:
            top_pre.remove((word,))
            
    #format collocations
    if len(top_post)>0:
        top_post=[str(top_post[i])[2:-3] for i in range(len(top_post))]
    else:
        top_post=["We cannot establish any probable post-positions from this corpus."]

        
    if len(top_pre)>0:
        top_pre=[str(top_post[i])[2:-3] for i in range(len(top_pre))]
    else:
        top_pre=["We cannot establish any probable pre-positions from this corpus."]
            
    #printing    
    print('WORD IN QUERY:','\033[1m'+string+'\033[0m')
    print('The most likely pre-positions are:'+'\033[1m')
    for i in top_pre:
        print(i)
    print("\033[0m"+'The most likely post-positions:'+'\033[1m')
    for i in top_post:
        print(i)

# Testing

Below I am testing a couple of words to see how my Collocations Dictionary work.

In [579]:
'''
Query a verb: 'have'.
'''
collocations(M,'have',words)

WORD IN QUERY: have
The most likely pre-positions are:
We cannot establish any probable pre-positions from this corpus.
The most likely post-positions:
use
calls


In [580]:
'''
Query a noun: 'arm'.
'''
collocations(M,'arm',words)

WORD IN QUERY: arm
The most likely pre-positions are:
We cannot establish any probable pre-positions from this corpus.
The most likely post-positions:
restrictions
chrysostomos
curling


In [581]:
'''
Query an adjective: "soft"
'''
collocations(M,'soft',words)

WORD IN QUERY: soft
The most likely pre-positions are:
We cannot establish any probable pre-positions from this corpus.
The most likely post-positions:
propped
shafts
meeting


## Evaluation

As the above tests show, my Collocation Dictionary is not yet perfect. It shows some justifiable collocations, such as "head shaking" or to "speak causally" but it also shows fragments of words and only loose associations. After playing with the code, and adjusting the different parameters of my model, I find that the problems might be:

1. Our base corpus is too small and specific. Instead of choosing a literary work of ~1.5 million words, we could utilise a larger corpus. 
2. I was not able to find an apporpiate skip-graph $n$ and $m$ value. What I found instead was that multiple specifications yield plausible results. Thus to finetune our results we must average the frequency for multiple specification of our model.
3. Incorporating linguistic knowledge in the system could be useful. Understanding the differences between nouns and adjectives, could allow us to create separate frequency tables that could filter for us better results than the current 'most frequent' entries approach.
4. Also, more sophisticated methods such as least-square regression could be implemented on our frequency matrix to find the best collocations.
5. I need to be more careful with string slicing, and word splitting. Clearly, I just got some words wrong. In hindsight I believe such a library as the Python NLTK could be useful.

All in all, this is not a bad start to this project. I was suprised to find that no-one has implemented this method specifically for the creation of a Collocation Dictionary. While not perfect, this project allowed me to explore the workings of n-grams, skip-grams and implement my own dynamic solution.

# An unexpected use case

While implementing my code, I realised that I could use the idea of my Markov matrix for statistical text generation. Given an input vector (a word), we can just use the matrix to find the most probable next word and thus put together sentences. In order to make the sentences a bit more realistic I used random weights for next word selection.

The results speak for themselves. Read below for some machine literature in the style of James Joyce.

In [493]:
'''
A faster than skip_gram(), non-dynamic n-gram builder.
'''
def generate_ngram(words,n=1):
    ngram = dict()

    # build ngram dict
    for i in range(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if key in ngram:
            ngram[key] += 1
        else:
            ngram[key] = 1

    #sort ngram dict by frequency
    ngram = sorted(ngram.items(), key=lambda count: count[1])
    ngram = ngram[::-1]
    return ngram

In [513]:
import random
'''
A random weighting function to introduce variation.
I: potential word choices
O: randomly selected word from potential choices
'''
def random_weight(potentials):
    total = sum(w for c, w in potentials)
    r = random.uniform(0, total)
    upto = 0
    for c, w in potentials:
        if upto + w > r:
            return c
        upto += w
'''
A simple sentence generator.
I: starting word, ngram, n
O: sentence of len n

'''
def generate_sentence(ngram, word, n = 30):
    sentence=''
    for i in range(n):
        sentence+=word+' '
        # Get all possible elements (( word 1, word2 etc.), frequency)
        potentials = [element for element in ngram if element[0][0] == word]
        if not potentials:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = random_weight(potentials)[1]
    print(sentence)
    
'''
Construct and concatenate a few sentences with different gram-lengths, and enjoy the results. 

'''    
    
gram2=generate_ngram(words,2)
gram3=generate_ngram(words,3)
gram4=generate_ngram(words,4)
gram5=generate_ngram(words,5)

t1= str(generate_sentence(gram2,'what'))
t2= str(generate_sentence(gram3,'they'))
t3= str(generate_sentence(gram4,'we'))
t4= str(generate_sentence(gram5,'and'))

print(t1,t2,t3,t4)


what spectacle it at all that exterior splendour of spinach dublin in twelve north and put her skirt a husband back garden whole course tomorrow eh i none other have 
they both full of fun gets their poetry well up in liontamer s so sad in the dentist money all that that girl s irish nation excellently commenced kissing smiling 
we are not solicit must really an infirm widow s queries dear bloom at doors rare lamps summer end of bronzefoil better find that aint half a peaceful not a 
and bacon a point was opened most private carr i ll square hats mr power announced on the voice production such thing done yeoman cap with lighted crevice of the 
None None None None


# References
1. Ruchirawat, N., & Ruchirawat, N. (2018, March 16). Collocations - identifying phrases that act like single words in Natural Language Processing. Retrieved from https://medium.com/@nicharuch/collocations-identifying-phrases-that-act-like-individual-words-in-nlp-f58a93a2f84a

2. An Introduction to N-grams: What Are They and Why Do We Need Them? (2017, October 21). Retrieved from https://blog.xrds.acm.org/2017/10/introduction-n-grams-need/

3. Oxford Collocations Dictionary. (n.d.). Retrieved from https://www.oxfordlearnersdictionaries.com/definition/collocations/
